Deal with unlabeled bbox (remove car IDs)
Goal: associate measurements to unique objects
1. try stitching objects in mutl-camera road

In [72]:
from utils import *
import importlib
import utils
importlib.reload(utils)
import os.path
from os import path
import pandas as pd
import utils_optimization as opt
importlib.reload(opt)
import data_association as da
importlib.reload(da)
import time
from scipy.optimize import minimize,NonlinearConstraint,leastsq,fmin_slsqp,least_squares
import numpy.linalg as LA
from shapely.geometry import Polygon

In [ ]:
# read and preprocess data
import utils
importlib.reload(utils)
data_path = pathlib.Path().absolute().joinpath('../June_5min')
tform_path = pathlib.Path().absolute().joinpath('../tform')
file_path = data_path.joinpath('p1c1_1_track_outputs_3D.csv')

df = utils.preprocess(file_path, tform_path, skip_row = 0)

In [67]:
# stitching based on prediction and overlaps

import utils
importlib.reload(utils)

df = utils.preprocess_data_association(df)


Before DA:  1144 cars
Frame :    0
Frame :  100
Frame :  200
Frame :  300
Frame :  400
Frame :  500
Frame :  600
Frame :  700
Frame :  800
Frame :  900
Frame : 1000
Frame : 1100
Frame : 1200
Frame : 1300
Frame : 1400
Frame : 1500
Frame : 1600
Frame : 1700
Frame : 1800
Frame : 1900
Frame : 2000
Frame : 2100
Frame : 2200
Frame : 2300
Frame : 2400
Frame : 2500
Frame : 2600
Frame : 2700
Frame : 2800
Frame : 2900
Frame : 3000
Frame : 3100
Frame : 3200
Frame : 3300
Frame : 3400
Frame : 3500
Frame : 3600
Frame : 3700
Frame : 3800
Frame : 3900
Frame : 4000
Frame : 4100
Frame : 4200
Frame : 4300
Frame : 4400
Frame : 4500
Frame : 4600
Frame : 4700
Frame : 4800
Frame : 4900
Frame : 5000
Frame : 5100
Frame : 5200
Frame : 5300
Frame : 5400
Frame : 5500
Frame : 5600
Frame : 5700
Frame : 5800
Frame : 5900
Frame : 6000
Frame : 6100
Frame : 6200
Frame : 6300
Frame : 6400
Frame : 6500
Frame : 6600
Frame : 6700
Frame : 6800
Frame : 6900
Frame : 7000
Frame : 7100
Frame : 7200
Frame : 7300
Frame : 7400
Fra

100%|██████████████████████████████████████████████████████████████████████████████| 8995/8995 [01:51<00:00, 80.85it/s]


In [70]:
# try rectification
import utils_optimization as opt
importlib.reload(opt)
df = opt.rectify(df) # parallel, batch method

100%|██████████████████████████████████████████████████████████████████████████████| 877/877 [2:35:11<00:00, 10.62s/it]


In [73]:
df = utils.post_process(df)

cap width at 2.59m...
standardize format for plotter...


In [74]:
df.to_csv('../June_5min/rectified/rectified_p1c1_uncorrected_5min.csv',index=False)

In [68]:
# make an animation based on LMCS
import os
import glob
import importlib
import animation_utils as an
importlib.reload(an)
image_folder = '../FramePic'

filelist = glob.glob(os.path.join(image_folder, "*"))
for f in filelist:
    os.remove(f)
    
if len(df['camera'].unique())==1:
    dim0 = get_camera_range(df['camera'][0])
else:
    dim0 = get_camera_range('all')
print(dim0)
dim = [d * 3.281 for d in dim0] # convert meter to feet
an.generate_frames(df, dim, skip_frame=1, image_folder=image_folder)

(0, 130, -5, 45)
Frame:  0 8994


In [69]:
# Fetch image files from the folder, and create an animation.
importlib.reload(an)
video_name = '../p1c1_uncorrected5min_DA.mp4'
an.write_video(image_folder, video_name, fps=30)

In [80]:
# visualize footprint on the camera video

import plot_rectified_objects
importlib.reload(plot_rectified_objects)

video = str(data_path.joinpath('raw_video/p1c1_1.mp4'))
label_file = str(data_path.joinpath('rectified/rectified_p1c1_uncorrected_5min.csv')) 
# # label_file = str(data_path.joinpath('rectified/p1_all/rectified_p1c3.csv')) 
# # label_file = str(data_path.joinpath('rectified/uncorrected_individual/rectified_p1c3_uncorrected.csv'))
# # label_file = str(data_path.joinpath('rectified/uncorrected_all/rectified_p1c3.csv'))
plot_rectified_objects.plot_vehicle_csv(video,label_file, frame_rate = 10,show_2d = False,show_3d = True,show_LMCS = True,show_rectified = False, ds=True)


In [78]:
label_file

'C:\\Users\\wangy79\\Documents\\I24_trajectory\\I24-trajectory-generation\\..\\June_5min\\rectified\\rectified_p1c1_uncorrected_5min.csv'

In [ ]:
# read & rectify each camera df individually
import utils
importlib.reload(utils)
data_path = pathlib.Path().absolute().joinpath('../June 2021 Data - 1 minute 5 cameras w RAV 4/')
tform_path = pathlib.Path().absolute().joinpath('../2D-3D May 2021/i24-roadway-transforms-main/tform')

# df = pd.DataFrame()
for root,dirs,files in os.walk(str(data_path), topdown = True):
    for file in files:
        if file.endswith(".csv") and file.startswith('p1c6'):
            
            # ... reading ...
            file_name = data_path.joinpath(file)
            camera_id = utils.find_camera_name(file)
            print('reading ',camera_id,'...')
            
            # ... preprocessing ...
            df = utils.preprocess(file_name, tform_path)
            df = df.assign(camera=camera_id)
            df = df.sort_values(by=['Frame #','Timestamp']).reset_index(drop=True)
        
            # ... rectifying ...
            df = opt.rectify(df)
            
            # ... post processing ...
            dfpp = utils.post_process(df)
            dfpp_img = utils.road_to_img(dfpp, tform_path)
            
            # ... saving ...
            new_name = 'rectified_' + camera_id +'.csv'
            dfpp_img.to_csv(new_name)
    break

In [ ]:
# plot 
import utils
importlib.reload(utils)
D = np.array(df[['bbr_x','bbr_y', 'fbr_x','fbr_y','fbl_x','fbl_y','bbl_x', 'bbl_y']])
utils.plot_track(D,15,5)

In [ ]:
fig, ax = plt.subplots()
# plt.xlim(xmin, xmax)
# plt.ylim(20, 42) # 0, 42
plt.xlabel('meter')
plt.ylabel('meter')
#plot design setup
plt.axhline(y = 21.25, color = 'black', linestyle = 'solid')
plt.axhline(y = 24, color = 'yellow', linestyle = '--')
plt.axhline(y = 18.5, color = 'yellow', linestyle = '--')
carColors=cm.rainbow(np.linspace(0,1,5))

# frameSnap = np.array(group[['bbr_x','bbr_y','fbr_x','fbr_y','fbl_x','fbl_y','bbl_x', 'bbl_y']])
for carid, track in tracks.items():
    frameSnap = track
    c = carColors[carid%5]
    for j in range(len(frameSnap)):  # draw one vehicle onto the plot
    #     carID = int(frameSnap[j,0])  # Finding the unique carID for each vehicle in the frame
        coord = frameSnap[j,0:8]   # Finding the coordinates of the car
        if (np.isnan(coord).any()):
            print('nan')
            continue
        coord = np.reshape(coord,(-1,2)).tolist()
        coord.append(coord[0])
        xs, ys = zip(*coord)
        plt.plot(xs,ys,c=c,label=carid if j==0 else '')
        #     plt.text(frameSnap[j,1], frameSnap[j,2], carID)
#     plt.legend()
